In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [2]:
!pip install ktrain

In [4]:
import tensorflow as tf
import pandas as pd
import numpy as np
import ktrain
from ktrain import text
import tensorflow as tf

In [5]:
tf.__version__

'2.3.1'

In [6]:
import os
print(os.listdir("../input/usi-nlp-practicum-2"))
import warnings
warnings.filterwarnings('ignore')

['glove.6B.50d.txt', 'winequality-red.csv', 'winequality-white.csv', 'imdb_train.csv', 'imdb_test_data.csv', 'sample_submission.csv']


In [7]:
#importing the training data
data_train = pd.read_csv('../input/usi-nlp-practicum-2/imdb_train.csv')
print(data_train.shape)
print(data_train.head(10))
#importing the training data
data_test=pd.read_csv('../input/usi-nlp-practicum-2/imdb_test_data.csv')
print(data_test.shape)
print(data_test.head(10))

(40000, 2)
                                              review sentiment
0  We had STARZ free weekend and I switched on th...  negative
1  I'll admit that this isn't a great film. It pr...  negative
2  I finally found a version of Persuasion that I...  positive
3  The BBC surpassed themselves with the boundari...  positive
4  Much praise has been lavished upon Farscape, b...  negative
5  Of course the plot, script, and, especially ca...  positive
6  This is one of those road movies that would li...  negative
7  What an uninteresting hodge-podge. It could ha...  negative
8  Only a handful of the segments are engaging he...  negative
9  THE GREEN BUTCHERS (Anders Thomas Jensen - Den...  positive
(10000, 2)
   id                                             review
0   1  Not only is this movie a great film for basic ...
1   2  Waitress: Honey, here's them eggs you ordered....
2   3  Many mystery stories follow the standard whodu...
3   4  A space ship cruising through the galaxy encou...


In [8]:
print("Size of train dataset: ",data_train.shape)
print("Size of test dataset: ",data_test.shape)


Size of train dataset:  (40000, 2)
Size of test dataset:  (10000, 2)


In [9]:
#sentiment count
data_train['sentiment'].value_counts()

positive    20048
negative    19952
Name: sentiment, dtype: int64

In [13]:

(X_train, y_train), (X_test, y_test), preproc = text.texts_from_df(train_df=data_train,
                                                                   text_column = 'review',
                                                                   label_columns = 'sentiment',
                                                                   val_df = data_test,
                                                                   
                                                                   maxlen = 500,
                                                                   preprocess_mode = 'bert')

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [15]:
X_test

[array([[  101,  2025,  2069, ...,     0,     0,     0],
        [  101, 13877,  1024, ...,  2003,  2055,   102],
        [  101,  2116,  6547, ...,     0,     0,     0],
        ...,
        [  101,  2108,  1037, ...,     0,     0,     0],
        [  101,  7929,  2034, ...,     0,     0,     0],
        [  101,  7910, 23644, ...,  2143,  1010,   102]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])]

In [16]:
model = text.text_classifier(name = 'bert',
                             train_data = (X_train, y_train),
                             preproc = preproc)

Is Multi-Label? False
maxlen is 500
done.


In [17]:
#here we have taken batch size as 6 as from the documentation it is recommend to use this with maxlen as 500

learner = ktrain.get_learner(model=model, train_data=(X_train, y_train),
                   val_data = (X_test, y_test),
                   batch_size = 6)

In [18]:
learner.fit_onecycle(lr = 2e-5, epochs = 1)

predictor = ktrain.get_predictor(learner.model, preproc)
predictor.save('bert')



begin training using onecycle policy with max lr of 2e-05...
6667/6667 [==============================] - 5667s 850ms/step - loss: 0.2230 - accuracy: 0.9091 - val_loss: 2.1429 - val_accuracy: 0.4996


In [25]:
import os
os.chdir(r'/kaggle/working')
predictor.save('/kaggle/working/bert')

!zip -r file.zip '/kaggle/working/'
print(os.listdir())
from IPython.display import FileLink
FileLink('kaggle/working/bert/tf_model.h5')

  adding: kaggle/working/ (stored 0%)
  adding: kaggle/working/__notebook_source__.ipynb (deflated 44%)
  adding: kaggle/working/bert/ (stored 0%)
  adding: kaggle/working/bert/tf_model.h5 (deflated 10%)
  adding: kaggle/working/bert/tf_model.preproc (deflated 52%)
['file.zip', '__notebook_source__.ipynb', 'bert']


In [27]:
from IPython.display import FileLink
FileLink('file.zip')

/kaggle/working/file.zip

In [28]:
data = ['this movie was horrible, the plot was really boring. acting was okay',
        'the fild is really sucked. there is not plot and acting was bad',
        'what a beautiful movie. great plot. acting was good. will see it again']

In [29]:
predictor.predict(data)

['negative', 'negative', 'positive']

In [44]:
dflist=data_test['review'].to_list()
data_test['prediction']=predictor.predict(dflist)
predictor.get_classes()

['negative', 'positive']

In [45]:
data_test.head()

,id,review,prediction
0,1,Not only is this movie a great film for basic ...,positive
1,2,"Waitress: Honey, here's them eggs you ordered....",negative
2,3,Many mystery stories follow the standard whodu...,positive
3,4,A space ship cruising through the galaxy encou...,negative
4,5,My favorite film this year. Great characters a...,positive


In [ ]:
import os
os.chdir(r'/kaggle/working')
predictor.save('/kaggle/working/bert')
data_test.to_excel('/kaggle/working/Prediction.xlsx')
!zip -r file.zip '/kaggle/working/'
print(os.listdir())
from IPython.display import FileLink
FileLink('file.zip')

test_data.to_excel('Output.xlsx')